# 🤖 ASR系列-探索-Whisper模型與結構

> 透過深入了解 Whisper 網路結構，提高學員對模型推理上的認知
> 日期：2025/10/08-0930

<a href="https://www.tenlong.com.tw/products/9786264142915"><img src="https://github.com/openpipe/art/raw/main/assets/Header_separator.png" height="5"></a>

## 👨‍💻 作者資源與聯絡方式

### 📚 深度學習專書
**📖 《LangGraph 實戰開發 AI Agent 全攻略》** - 我的最新技術著作
深入探討 LangGraph、Agentic AI System 等前沿技術
**[立即購買](https://www.tenlong.com.tw/products/9786264142915)**

### 🌐 社群媒體與技術交流
如果您有任何疑問或想要進一步交流，歡迎透過以下管道聯絡：

* **📖 技術專書**： [購買我的 LangGraph 實戰開發 AI Agent 全攻略](https://www.tenlong.com.tw/products/9786264142915)
* **💻 GitHub**： [我的開源專案](https://github.com/Heng-xiu)
* **🤗 Hugging Face**： [我的模型與資料集](https://huggingface.co/Heng666)
* **✍️ 部落格**： [技術文章分享](https://r23456999.medium.com/)

感謝大家的支持！期待與更多 AI 技術愛好者交流討論 🚀

<div class="align-center">
  <a href="https://ko-fi.com/hengshiousheu"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a>
</div>

<a href="https://www.tenlong.com.tw/products/9786264142915"><img src="https://github.com/openpipe/art/raw/main/assets/Header_separator.png" height="5"></a>



#第一章、Whisper ASR 模型

##1.1-教學流程

本教學將逐步指導您熟悉與了解 Whisper 運作機制

涵蓋以下核心步驟

1. 模型結構機制
2. Encoder
3. Decoder
4. 多任務學習
5. 體驗模型效果



---



# 第二章：環境建置與前置準備
在我們開始微調 Whisper 之前，首先要確保我們的開發環境已經準備就緒。一個穩定且配置正確的環境是成功訓練模型的第一步。

> GPU 啟用： 請確保您的 Colab Notebook 已啟用 GPU。點擊菜單欄的 執行階段 (Runtime) -> 變更執行階段類型 (Change runtime type)，然後在 硬體加速器 (Hardware accelerator) 中選擇 GPU。

##2.1 首先，來登入 HuggingFace

本章節我們先確認在本實驗環境中，可以獲取到 HuggingFace 資源，包含下載資料集、模型等操作

您可以在 Hugging Face Hub 中找到您的 [Hugging Face token](https://huggingface.co/login?next=%2Fsettings%2Ftokens)


In [1]:
from google.colab import userdata
from huggingface_hub import HfApi

HF_TOKEN = userdata.get("HF_TOKEN")

api = HfApi(token=HF_TOKEN)
username = api.whoami()['name']
print(username)

Heng666


## 2.2 GPU 驅動與 CUDA 支援確認

LLM 的訓練需要大量的計算資源，幾乎必須仰賴 GPU (Graphics Processing Unit)。因此，確認您的環境是否正確偵測到 GPU 並支援 CUDA 是至關重要的一步。

CUDA 是 NVIDIA 提供的平行運算平台和程式設計模型，允許軟體使用 GPU 進行通用計算。PyTorch (一個流行的深度學習框架) 透過 CUDA 來利用 NVIDIA GPU 的運算能力。

請執行以下 Python 程式碼，檢查您的 PyTorch 環境是否已正確偵測到 CUDA：

In [2]:
import torch

print(f"PyTorch 是否支援 CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"目前使用的 CUDA 裝置名稱: {torch.cuda.get_device_name(0)}")
    print(f"CUDA 裝置數量: {torch.cuda.device_count()}")
    # 可以進一步檢查 CUDA 版本
    print(f"PyTorch 編譯的 CUDA 版本: {torch.version.cuda}")
    # 執行 nvidia-smi (僅限 Linux/Windows 終端機，Colab 可直接執行)
    # !nvidia-smi
else:
    print("警告：未偵測到 CUDA。模型訓練將在 CPU 上運行，速度會非常慢。")
    print("請檢查您的 GPU 驅動程式安裝、CUDA Toolkit 設定以及 PyTorch 的 CUDA 支援。")

PyTorch 是否支援 CUDA: True
目前使用的 CUDA 裝置名稱: Tesla T4
CUDA 裝置數量: 1
PyTorch 編譯的 CUDA 版本: 12.6


> 【重要提醒】：如果 torch.cuda.is_available() 回傳 False，您可能需要：

確認您的電腦有 NVIDIA GPU。

安裝正確版本的 NVIDIA 顯示卡驅動程式。

安裝與您 PyTorch 版本相容的 CUDA Toolkit。

對於 Colab 用戶，請再次確認您已在執行階段中選擇了 GPU。



---



#第三章、深入淺出 Whisper ASR 模型結構


![img](https://raw.githubusercontent.com/openai/whisper/main/approach.png)
https://raw.githubusercontent.com/openai/whisper/main/approach.png


##3.1- Encoder-負責項目

Encoder 的角色是將原始音頻轉換為高階特徵表示（hidden representations），捕捉音頻的語義和模式（如聲音的頻率、節奏）。輸入是我們之前討論的 Mel-spectrogram（梅爾頻譜圖）。

Encoder 拿音頻的 Mel-spectrogram 當輸入，這是個 2D 圖，像聲音的熱圖，80 個頻率通道，30 秒有 3000 幀。先用兩個卷積層壓縮時間維度到大約 1500 幀，提取低階特徵。然後丟進 Transformer 層，通過 self-attention 捕捉音頻的全局模式，比如語音的節奏或音調。最後輸出一個高維特徵序列，像是音頻的『語義壓縮版』，準備給 decoder 用。

> Encoder 只處理音頻，不涉及文本。它輸出的是一組 "normalized audio feature representations"（歸一化的音頻特徵向量），代表音頻的壓縮和高抽象表示。

##3.2- Decoder

Decoder 是生成文本的機器，從一個起始 token（像 <|startoftranscript|>）開始，逐步猜下一個字。它用 self-attention 看自己已經生成的文本，然後用 cross-attention 去『問』 encoder 的音頻特徵，找對應的聲音內容。Transformer 層處理完後，輸出下一個 token 的概率，重複直到生成完整句子。訓練時，我們給它正確文本，逼它學會把音頻和文字對起來。

> Decoder 像一個條件語言模型：它 "聽" 音頻（通過 cross-attention），生成對應文本。推理時用 beam search 或 greedy decoding；訓練時用 teacher-forcing（用 ground truth token 作為輸入）。

##3.3-Encoder–Decoder 資料流與 token 流程

Whisper 的核心是 Encoder–Decoder Transformer。
我們從聲音波形開始，把 16kHz 的 waveform 經過梅爾頻譜（Mel-spectrogram）轉成一張「時間 × 頻率」的特徵圖。這張特徵圖（約 3000×80）被送進 Encoder。

Encoder 的角色就像“聽覺理解模組”，透過多層自注意力機制，把聲音壓縮成語音語意向量（Audio Embeddings），輸出形狀約為 (T_enc, d_model)，例如 (1500, 512)。

---

接著 Decoder 開始「說話」。它不是直接接收聲音，而是看兩個東西：
1. 自己已經生成的文字 token（Self-Attention）。
2. Encoder 給的語音語意向量（Cross-Attention）。

---
每一步 Decoder 都會根據這兩個資訊，預測下一個 token。
例如它看到 <|startoftranscript|> + <|zh|> + <|transcribe|> + <|notimestamps|> 後，注意 Encoder 的音訊向量，預測出下一個中文字 token。


整個過程持續到模型輸出 <|endoftext|> 為止。
所以 Whisper 不是分類器，而是一個「條件語言模型」：條件是音訊的 Encoder 表徵。

In [1]:
!pip install --quiet datasets==3.6.0
!pip install --quiet torchcodec==0.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.3 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

minds = load_dataset("PolyAI/minds14", "zh-CN",split="train")
from datasets import Audio
# 確保音訊的取樣率一致（例如 16 kHz）
minds = minds.cast_column("audio", Audio(sampling_rate=16000))

# 取得第一個音訊範例
audio = minds[0]["audio"]

README.md: 0.00B [00:00, ?B/s]

zh-CN/train-00000-of-00001.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/502 [00:00<?, ? examples/s]

In [3]:
import torch
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

checkpoint = "openai/whisper-tiny"
processor = AutoProcessor.from_pretrained(checkpoint)
model = AutoModelForSpeechSeq2Seq.from_pretrained(checkpoint)
model.eval()

# 假設已有音訊 waveform
inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt")
input_features = inputs.input_features

# 1️⃣ Encoder 輸出
with torch.no_grad():
    encoder_out = model.model.encoder(input_features)
print("Encoder output shape:", encoder_out.last_hidden_state.shape)

# 2️⃣ Decoder 起始提示
forced_decoder_ids = processor.get_decoder_prompt_ids(
    language="zh",
    task="transcribe",
    no_timestamps=True
)
print("Decoder forced tokens:",
      [processor.tokenizer.decode([tid]) for _, tid in forced_decoder_ids])

# 3️⃣ Decoder 輸入與前向傳遞
decoder_input_ids = torch.tensor([[processor.tokenizer.bos_token_id]])
with torch.no_grad():
    decoder_out = model.model.decoder(
        input_ids=decoder_input_ids,
        encoder_hidden_states=encoder_out.last_hidden_state
    )
print("Decoder hidden state shape:", decoder_out.last_hidden_state.shape)

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

Encoder output shape: torch.Size([1, 1500, 384])
Decoder forced tokens: ['<|zh|>', '<|transcribe|>', '<|notimestamps|>']
Decoder hidden state shape: torch.Size([1, 1, 384])


##3.4-多任務學習


###3.4.1-根據模型圖片說明「多任務」機制（Whisper 概念面）

Whisper 以 **Encoder–Decoder Transformer** 將一段最長約 30 秒的音訊（16 kHz）轉為 **3000×80 的 log-Mel**，餵入 **Encoder** 萃取「語音語義向量」；**Decoder** 端透過「**特殊 token 序列**」決定任務與輸出形式，並以 **cross-attention** 對齊音訊表徵來逐步預測文字 token。
四類任務共用同一模型，以**不同的起始提示序列（prompt tokens）**區分：

* **英語轉錄（English → English）**：`<|startoftranscript|> <|en|> <|transcribe|> [<|notimestamps|> 或時間戳設定] ... → EOT`
* **多語轉錄（X → X）**：`<|startoftranscript|> <|zh|> <|transcribe|> ... → EOT`
* **多語翻譯（X → English）**：`<|startoftranscript|> <|zh|> <|translate|> ... → EOT`
* **無語音偵測（No-Speech）**：解碼初期會估計 `no_speech` 機率，若高於門檻可視為「只有背景，無人說話」。

重點在於：**任務不是在 Encoder 分支選擇**，而是 **Decoder 以不同的 SOT + 語言 + 任務 token** 來「告訴」模型該做什麼；這讓同一組權重在不同提示下完成不同任務。

###3.3.2-對照 🤗 Transformers 檔案位置（實作面有感）

> 你目前看的 `configuration_whisper.py` 是「結構超參數」；多任務行為主要發生在 **產生解碼提示** 與 **強制起始 token** 這兩處。

* **`generation/configuration_whisper.py` — `WhisperGenerationConfig`**

  * 角色：行為設定（*不是*網路結構）。
  * 重要欄位：`task`（"transcribe"/"translate"）、`language`、`no_timestamps` 等。這些欄位用來**生成 `forced_decoder_ids`**（強制在解碼步 0..k 插入指定 token 序列）。
* **`modeling_whisper.py` — `WhisperForConditionalGeneration.get_decoder_prompt_ids`**

  * 角色：**把上面的任務設定→具體 token 序列**。
  * 作法：根據 `generation_config.task/language/no_timestamps`，組出 `[(pos0, SOT), (pos1, <|lang|>), (pos2, <|transcribe|> or <|translate|>), ...]`，回傳給 `generate()` 用來固定起始解碼內容。
  * 這裡就是「多任務落地」的關鍵。
* **`tokenization_whisper.py`／`processing_whisper.py` — Token 與前處理**

  * 定義 `<|startoftranscript|>、<|translate|>、<|transcribe|>、<|notimestamps|>` 等 special tokens；`Processor` 將 **log-Mel** 與 **labels** 封裝成張量。
* **`modeling_whisper.py` — `generate()` 流程**

  * 在 beam/search 迴圈中讀取 `forced_decoder_ids` 先行寫入起始步，之後才開始自回歸預測下一個 token。
* **（近似 VAD）`no_speech` 機率**

  * 推論過程會輸出 `no_speech_prob`，常搭配 chunking + threshold 當作「無語音」的啟發式判斷；不是獨立 VAD 子網路。


###3.3.3-小結：
>
> * **結構**在 `configuration_whisper.py`；
> * **行為（多任務）**由 `WhisperGenerationConfig` + `get_decoder_prompt_ids` 決定；
> * **資料**端只需提供 `(audio, text)` 配對；你要「轉錄」或「翻譯」由 **解碼提示**與**標籤語言**共同決定。

##3.4.-呼叫模型進行一次推論



In [4]:
import torch
import numpy as np
from transformers import (
    AutoProcessor,
    AutoModelForSpeechSeq2Seq,
    WhisperTokenizer,
    WhisperFeatureExtractor,
    set_seed,
)

set_seed(42)

audio_array = minds[0]["audio"]["array"]
sampling_rate = minds[0]["audio"]["sampling_rate"]
assert sampling_rate == 16000, f"Whisper 預設 16kHz，當前為 {sampling_rate}"

幸運的是，🤗 Transformers Whisper 特徵提取器僅用一行程式碼即可執行填充和聲譜圖變換兩個操作！我們使用以下程式碼從預先訓練的 checkpoint 中載入特徵提取器，為音訊資料處理做好準備:

##3.4.1-載入處理器與模型（可換成你要的 checkpoint，如 'openai/whisper-small'）

In [5]:
# 1)
checkpoint = "openai/whisper-tiny"
processor = AutoProcessor.from_pretrained(checkpoint)  # 封裝 tokenizer + feature_extractor
model = AutoModelForSpeechSeq2Seq.from_pretrained(checkpoint)
model.eval()

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 384)
      (layers): ModuleList(
        (0-3): 4 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=384, out_features=384, bias=False)
            (v_proj): Linear(in_features=384, out_features=384, bias=True)
            (q_proj): Linear(in_features=384, out_features=384, bias=True)
            (out_proj): Linear(in_features=384, out_features=384, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (fin

###3.4.2-轉換 input_features

In [6]:
# 2) waveform -> log-Mel input_features（Whisper 風格）
inputs = processor.feature_extractor(
    audio_array,
    sampling_rate=16000,
    return_tensors="pt",
)
input_features = inputs.input_features  # 形狀 ~ [batch, frames, mel_bins] ⇒ [1, 3000, 80]（視長度而變）
print("input_features.shape:", tuple(input_features.shape))

input_features.shape: (1, 80, 3000)


現在我們載入 Whisper 分詞器。 Whisper 模型會輸出詞元，這些字元表示預測文本在字典中的索引。分詞器負責將這一系列詞元映射為最終的文本字串(例如[1169, 3797, 3332] -> “the cat sat”)。


Whisper 分詞器在 96 種語種資料上預先訓練而得，因此，其位元組對(byte-pair) 覆蓋率很廣，幾乎包含了所有語種。就中文而言，我們可以載入分詞器並將其直接用於微調。只需指定目標語種和任務，分詞器就會根據這些參數將語種和任務標記加為輸出序列的前綴:

###3.4.3-從 processor 取得「多任務提示 token 序列」

In [7]:
# 3)
prompt = processor.get_decoder_prompt_ids(
    language="zh",          # 中文
    task="transcribe",      # 轉錄
    no_timestamps=True      # 不輸出時間戳；若要時間戳，改成 False
)
print("forced_decoder_ids:", prompt)
print([processor.tokenizer.decode([tid]) for _, tid in prompt])

forced_decoder_ids: [(1, 50260), (2, 50359), (3, 50363)]
['<|zh|>', '<|transcribe|>', '<|notimestamps|>']


直接傳給 generate 函數

In [8]:
gen_ids = model.generate(input_features=input_features, forced_decoder_ids=prompt, max_new_tokens=225)

Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


###3.4.4-Decode token

In [9]:
# 4) 解碼
text = processor.tokenizer.batch_decode(gen_ids, skip_special_tokens=True)[0]
print(text)

那好想要了解一下我的銀行贊互約有多少謝謝
